# Modelos de N-gramas

Modelos de N-gramas são modelos de linguagem probabilísticos em que assumimos uma versão generalizada da condição de Markov. Assim, para predizer a n-ésima palavra, procuramos maximizar a probabilidade condicional $P(w_n|w_1, \cdots,w_{n-1})$

## Materiais

[Módulo NLTK -  Modelos de Linguagem](https://www.nltk.org/api/nltk.lm.html)

In [1]:
import nltk
nltk.download("machado")
nltk.download('punkt')
from nltk.corpus import machado

[nltk_data] Downloading package machado to
[nltk_data]     C:\Users\oknotok\AppData\Roaming\nltk_data...
[nltk_data]   Package machado is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\oknotok\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


O módulo lm do nltk oferece diversas implementações de modelos probabilísticos de linguagem e ferramentas utilitárias para calcular n-gramas, preparar dados para o treinamento de modelos, etc.

In [2]:
from nltk.lm.preprocessing import pad_both_ends
from nltk.lm.preprocessing import flatten
from nltk.util import bigrams,everygrams
print(machado.sents()[:10])
print(list(bigrams(flatten(machado.sents()[:10]))))
# print(list(everygrams(flatten(machado.sents()[:10]), max_len=3, min_len=3)))
print(list(everygrams(flatten(machado.sents()[:10]), max_len=3)))


[['Conto', ',', 'Contos', 'Fluminenses', ',', '1870'], ['Contos', 'Fluminenses'], ['Texto', '-', 'fonte', ':'], ['Obra', 'Completa', ',', 'Machado', 'de', 'Assis', ',', 'vol', '.'], ['II', ','], ['Rio', 'de', 'Janeiro', ':', 'Nova', 'Aguilar', ',', '1994', '.'], ['Publicado', 'originalmente', 'pela', 'Editora', 'Garnier', ',', 'Rio', 'de', 'Janeiro', ',', 'em', '1870', '.'], ['ÍNDICE'], ['MISS', 'DOLLAR'], ['LUÍS', 'SOARES']]
[('Conto', ','), (',', 'Contos'), ('Contos', 'Fluminenses'), ('Fluminenses', ','), (',', '1870'), ('1870', 'Contos'), ('Contos', 'Fluminenses'), ('Fluminenses', 'Texto'), ('Texto', '-'), ('-', 'fonte'), ('fonte', ':'), (':', 'Obra'), ('Obra', 'Completa'), ('Completa', ','), (',', 'Machado'), ('Machado', 'de'), ('de', 'Assis'), ('Assis', ','), (',', 'vol'), ('vol', '.'), ('.', 'II'), ('II', ','), (',', 'Rio'), ('Rio', 'de'), ('de', 'Janeiro'), ('Janeiro', ':'), (':', 'Nova'), ('Nova', 'Aguilar'), ('Aguilar', ','), (',', '1994'), ('1994', '.'), ('.', 'Publicado'), (

Comumente, na preparação dos textos, marcamos o início e final de cada sentença com marcadores especiais para delimitar o início e fim da sentença

In [3]:
from nltk.lm.preprocessing import pad_both_ends
padded_sents = list(pad_both_ends(machado.sents()[0],n=2))
print(padded_sents)
bigramas = list(bigrams(padded_sents))
print(bigramas)

['<s>', 'Conto', ',', 'Contos', 'Fluminenses', ',', '1870', '</s>']
[('<s>', 'Conto'), ('Conto', ','), (',', 'Contos'), ('Contos', 'Fluminenses'), ('Fluminenses', ','), (',', '1870'), ('1870', '</s>')]


Vamos treinar um modelo de maximização da verossimilhança esperada (MLE)

In [4]:
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm.models import MLE
import re
from sklearn.model_selection import train_test_split
files = [fileid for fileid in machado.fileids() if re.match("romance",fileid)]
data = machado.sents(fileids=files)
train_data, test_data = train_test_split(data,test_size=0.3)
train, vocab = padded_everygram_pipeline(2, train_data)
lm = MLE(2)

lm.fit(train,vocab)

Vamos investigar um pouco o que está sendo representado nesse modelo.
   - O atributo counts do modelo armazena o dicionário que associa a cada contexto uma distribuição de probabilidade discreta sobre as palavras do vocabulario
   - Podemos então calcular a probabilidade de uma palavra (ou de uma palavra dado um contexto) a partir dele

In [8]:
print(lm.counts)
print(lm.counts['Conto'])
print("P('cigana')=",lm.score("cigana"))
print("P('cigana'|'olhos','de')=",lm.score("cigana", ["olhos","de"]))

print(len(lm.vocab))

<NgramCounter with 3 ngram orders and 1006029 ngrams>
7
P('cigana')= 3.875045289591822e-06
P('cigana'|'olhos','de')= 0
26700


Note que a expressão "olhos de cigana" aparece com pouquíssima frequência no texto e portanto, a probabilidade $P(\mbox{cigana}|\mbox{olhos},\mbox{de})$ é muito baixa.

Dado um conjunto de dados de teste, podemos também avaliar o modelo treinado com base nesses dados através da métrica de perplexidade

In [ ]:
# lm.perplexity(test_data)

from nltk.lm.preprocessing import padded_everygram_pipeline
test_set = list(padded_everygram_pipeline(2, (test_data)))
lm.perplexity(everygrams(flatten(test_data), 2))

In [10]:
from collections import *
from random import random
def normalize(counter):
   s = float(sum(counter.values()))
   return [(c,cnt/s) for c,cnt in counter.items()]


def train_char_lm(texts, order=4):
    lm = defaultdict(Counter)
    for text in texts:
       data = text
       pad = "~" * order
       data = pad + data + ("</s>"*order)
       for i in range(len(data)-order):
         for j in range(1, order):
           history, char = data[i:i+j], data[i+j]
           lm[history][char]+=1
    outlm = {hist:normalize(chars) for hist, chars in lm.items()}
    return outlm



In [11]:
def generate_letter(lm, history, order):
        history = history[-order:]
        dist = []
        while dist == [] and history != '':
           try:
              dist = lm[history]
           except:
              history = history[1:]
        x = random()
        for c,v in dist:
            x = x - v
            if x <= 0: return c

In [12]:
def generate_text(lm, order, nletters=1000):
    history = "~" * order
    out = []
    for i in range(nletters):
        c = generate_letter(lm, history, order)
        history = history[-order:] + c
        out.append(c)
    return "".join(out)

In [13]:
import nltk
nltk.download("machado")

[nltk_data] Downloading package machado to
[nltk_data]     C:\Users\oknotok\AppData\Roaming\nltk_data...
[nltk_data]   Package machado is already up-to-date!


True

In [14]:
from nltk.corpus import machado
import re
order = 15
files = [fileid for fileid in machado.fileids() if re.match("romance/",fileid)]
texts = [machado.raw(fileids=[file]) for file in files]
lm = train_char_lm(texts, order=order)

In [15]:

text = generate_text(lm, order, nletters=5000)
print(text)

~ROMANCE, Memorial de Aires,1908

Memorial de
Aires

Texto-fonte:
Obra Completa, Machado de
Assis,
Rio
de Janeiro: Editora
  Nova Aguilar, Rio de Janeiro, 1904.

ADVERTÊNCIA DA PRIMEIRA
  EDIÇÃO

Não sei o que ela disse a Natividade? Não fez mais que ameaçá-la com palavras de tristeza. Não vinha alegre, a filha, que ela
morreria talvez
se me calasse, do mesmo modo...

-- De outro modo, ambos eles podiam ser felizes.

Meneses repeliu a idéia de fazer
  confidência da mãe.
  A impressão foi profunda e dolorosa, mas o sentido era este. E Fidélia deixaria
a mesa sem chorar, como Pedro chorou depois do galo.

Tudo imaginações! Crede-me: há tiranos de
intenção. Quem sabe? Não conhecia
  a hipocrisia, a perfídia da serpente e da
desobediência do homem, ainda a engraxar botas, é sublime.

CAPÍTULO XVIII / VISÃO DO
CORREDOR

No fim da escada, ao fundo do
corredor escuro, parei alguns instantes de silêncio, ergueu-se e foi
  dali ao gabinete de Estácio.
  Será ela que está ali, com uma senhora i

In [16]:
def generate_from_prompt(lm, prompt, order, nletters=1000):
    history = "~" * (order-len(prompt)) + prompt[-order:]
    out = []
    out.extend(prompt)
    for i in range(nletters):
        c = generate_letter(lm, history, order)
        history = history[-order+1:] + c
        out.append(c)
    return "".join(out)

In [29]:
prompt = "ao verme"
text = generate_from_prompt(lm,prompt,order)
print(text)


ao verme
que
primeiro roeu as frias
carnes
do meu cadáver

dedico
como saudosa lembrança

estas
Memórias
Póstumas

Prólogo da terceira
edição

A primeira edição destas
Memórias Póstumas de Brás Cubas, descobri uma
lei sublime, a lei da equivalência das janelas, e estabeleci que o modo de
compensar uma janela fechada;
eu abri outra com o gesto de
aproximação. Certo é que Capitu recuou um pouco para deixá-la
passar. Os olhos com que o buscava, os apertos de mão
significativos e
  eloqüentes. Não era recente a afeição dela; era talvez a parte mais enfadonha do sistema,
posto que concebida com um formidável rigor de lógica. Reorganizada a sociedade
pelo método dele, nem por isso a história morre.

Resta saber (é o ponto escuro)
como é que Santos pôde calar por longos dias um negócio tão importante para ele, como a procurar por
onde começaria. Quando os levantou deu com a inglesa. Ia já falar, mas estacou.
A afeição que lhe tinha não impediu que achasse demasiada familiaridade que existia e

In [31]:
%pip install datasets==2.14.7
%pip install pyarrow==16.1.0

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import torch
import torch.nn as nn
import pyarrow as pa

class CharLM(nn.Module):
   def __init__(self,vocab,order, hidden_size=100,num_layers=3,dropout = 0.5):
      super().__init__()
      if len(vocab)<=1:
         raise Exception("Vocabulário deve ter pelo menos dois símbolos")
      self.id2voc ={0:"<UNK>", 1:'<PAD>',2:"<s>",3:"</s>"}
      self.id2voc.update({i+3:c for i,c in enumerate(vocab)})
      self.voc2id = {self.id2voc[key]:key for key in self.id2voc}
      self.hidden_size = hidden_size
      self.input_size = order
      self.lstm = nn.LSTM(self.input_size,self.hidden_size,num_layers=num_layers,batch_first=True)
      self.dropout = nn.Dropout(p=dropout)
      self.out_head = nn.Linear(self.hidden_size,len(self.id2voc))
      self.relu = nn.ReLU()
   def forward(self,contexts,labels=None):
      if len(contexts.shape) == 1:
         contexts=contexts.unsqueeze(dim=0)
      loss_fn = nn.CrossEntropyLoss()
      encoded = self.lstm(contexts)[0]
      if self.train:
         encoded = self.dropout(encoded)
      logits = self.relu(self.out_head(encoded))
      loss = None
      if labels!=None:
         loss = loss_fn(logits,labels)
      if loss:
         return (loss,torch.multinomial(logits.softmax(dim=-1), num_samples=1))
      else:
         return torch.multinomial(logits.softmax(dim=1),num_samples=1)
   def predict(self,text,n_letters=1):
      device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
      outputs = list(text)
      inputs = [float(self.voc2id["<PAD>"])]*(self.input_size-len(text)-1)
      if len(text)<self.input_size:
         inputs;append(float(self.voc2id["<s>"]))
      inputs.extend([float(self.voc2id[c]) if c in self.voc2id else float(self.voc2id["<UNK>"])
                        for c in outputs[-self.input_size:]
                    ])
      for i in range(n_letters):
         predicted = self.forward(torch.tensor(inputs).to(device))[0]
         outputs.append(self.id2voc[predicted.item()])
         inputs = inputs[1:]
         inputs.append(float(predicted))
      return "".join(outputs)
   def convert_texts_to_contexts(self,texts):
      contexts = []
      labels = []
      for text in texts:
         converted = [self.voc2id["<PAD>"]]*(self.input_size-1)
         converted.append(self.voc2id["<s>"])
         converted.extend([self.voc2id[c] if c in self.voc2id else self.voc2id['<UNK>']
                              for c in list(text)
                          ])
         converted.extend([self.voc2id["</s>"]]*self.input_size)
         for i in range(len(converted)-self.input_size):
            contexts.append([float(i) for i in converted[i:i+self.input_size]])
            labels.append(converted[i+self.input_size])
      return pa.table([contexts,labels],names=["contexts","labels"])


In [ ]:
def create_vocab(texts):
   text = "".join(texts)
   return set(list(text))


In [ ]:
from nltk.corpus import machado
import re

files = [fileid for fileid in machado.fileids() if re.match("romance/",fileid)]
texts = [machado.raw(fileids=[file]) for file in files]
order = 15
vocab = create_vocab(texts)
model = CharLM(vocab,order)

In [ ]:
from datasets import Dataset
dataset = Dataset(model.convert_texts_to_contexts(texts)).with_format(type="torch")
dataset

Dataset({
    features: ['contexts', 'labels'],
    num_rows: 3098589
})

In [ ]:
dataset[0]

{'contexts': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'labels': tensor(32)}

In [ ]:
from transformers import TrainingArguments, Trainer
training_args =  TrainingArguments(
          output_dir = "./",
          num_train_epochs = 0.25,
          eval_strategy= 'no',
          auto_find_batch_size=True,
          remove_unused_columns=True,
      )

trainer = Trainer(
       model=model,
       args=training_args,
       train_dataset=dataset,
   )

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss


ValueError: Trainer: evaluation requires an eval_dataset.

In [32]:
from transformers import AutoModel, AutoTokenizer
model_name = "neuralmind/bert-base-portuguese-cased"
tokenizer =  AutoTokenizer.from_pretrained(model_name)
transf_model = AutoModel.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

c:\Python311\Lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\oknotok\.cache\huggingface\hub\models--neuralmind--bert-base-portuguese-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/647 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [33]:
sent = " ".join(machado.raw().split('\n')[56:67])
print(sent)
print(tokenizer.tokenize(sent,add_special_tokens=True))


 Era conveniente ao romance que o leitor ficasse muito tempo sem saber quem era Miss Dollar. Mas por outro lado, sem a apresentação de Miss Dollar, seria o autor obrigado a longas digressões, que encheriam o papel sem adiantar a ação. Não há hesitação possível: vou apresentar-lhes Miss Dollar.  Se o leitor é rapaz e dado ao gênio melancólico, imagina que Miss Dollar é uma inglesa pálida e delgada, escassa de carnes e de sangue, abrindo à flor do rosto dois grandes olhos azuis e sacudindo ao vento umas longas tranças loiras. A moça em questão deve ser
['[CLS]', 'Era', 'conveniente', 'ao', 'romance', 'que', 'o', 'leitor', 'fica', '##s', '##se', 'muito', 'tempo', 'sem', 'saber', 'quem', 'era', 'Miss', 'Dol', '##lar', '.', 'Mas', 'por', 'outro', 'lado', ',', 'sem', 'a', 'apresentação', 'de', 'Miss', 'Dol', '##lar', ',', 'seria', 'o', 'autor', 'obrigado', 'a', 'longas', 'dig', '##ress', '##ões', ',', 'que', 'ench', '##eria', '##m', 'o', 'papel', 'sem', 'adia', '##ntar', 'a', 'ação', '.', 'N

In [34]:
encoded = tokenizer(sent,return_tensors='pt')
print(encoded)
print(encoded['input_ids'].shape)

{'input_ids': tensor([[  101,  3362, 22020,   320,  4313,   179,   146, 14624,  1968, 22281,
           236,   785,   596,   834,  4945,  1977,   495,  5332, 10403,  7177,
           119,  1645,   240,  1342,  1341,   117,   834,   123,  4689,   125,
          5332, 10403,  7177,   117,  1467,   146,  1368,  9773,   123,  9800,
          2826,  1239,   270,   117,   179, 16386,  1994, 22287,   146,  1798,
           834,  8684,  2443,   123,  2973,   119,  2542,  1307, 18540,  3489,
          2199,   131, 17891,  4849,   118,  7707,  5332, 10403,  7177,   119,
           530,   146, 14624,   253, 13254,   122,  3433,   320,  2659,   247,
           949,  1601, 12658,   117, 12223, 22278,   179,  5332, 10403,  7177,
           253,   230,  6369,  3097,  3093,   285,   122,  2607,  3281,   117,
          3240, 22281,   375,   125,  7714, 22281,   122,   125,  5052,   117,
         14094,   353, 12252,   171,  9169,   682,  1491,  5708, 16467,   122,
           629,   830,  6436,   243,  

In [35]:
output = transf_model(**encoded)
output

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.3321, -0.0222, -0.1118,  ..., -0.2713, -0.0577, -0.3856],
         [ 0.4673, -0.6415,  0.0684,  ...,  0.0043,  0.4092, -0.5008],
         [ 0.4127, -0.2568,  0.3349,  ...,  0.6755, -0.1253, -0.1977],
         ...,
         [-0.0346,  0.1697,  0.2855,  ..., -0.4704, -0.0712, -0.1308],
         [ 0.3084, -0.2054,  0.7561,  ..., -0.2902, -0.0521, -0.8002],
         [ 0.3327, -0.0211, -0.1130,  ..., -0.2712, -0.0568, -0.3902]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[-2.3519e-02,  1.7835e-03, -1.5374e-01,  9.1325e-02,  3.3067e-02,
         -1.4718e-01,  8.2286e-01, -1.2356e-01,  1.4054e-01, -1.9458e-01,
         -4.6636e-01, -7.8418e-03,  1.7339e-01, -4.2789e-02, -1.7140e-01,
          9.2816e-02, -3.6222e-02, -1.0855e-01, -1.5493e-01,  8.5959e-01,
          1.5545e-02, -5.9975e-02,  1.2371e-01, -1.7211e-01, -1.3424e-01,
         -1.8552e-01,  6.0686e-02,  2.8905e-02,  1.4735e-01, -1.760

In [36]:
print(output.last_hidden_state.shape)

torch.Size([1, 140, 768])


In [37]:
from transformers import AutoModelForMaskedLM
lm = AutoModelForMaskedLM.from_pretrained(model_name)



Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [38]:
text = "O rato roeu a [MASK] do rei de Roma"
encoded = tokenizer(text,return_tensors='pt')
print(encoded['input_ids'])


tensor([[  101,   231,   646,   183,   577, 13665,   123,   103,   171,  1754,
           125,  2108,   102]])


In [42]:
ids = lm(**encoded)
print(ids)
ids = ids.logits
print(ids.shape)
ids = ids.squeeze(dim=0)
print(ids.shape)
ids = ids.softmax(dim=1).argmax(dim=1)
print(ids)
tokenizer.decode(ids[1:-1])

MaskedLMOutput(loss=None, logits=tensor([[[ -7.0549,  -7.4780,  -7.6611,  ...,  -7.5475,  -6.3172,  -6.5222],
         [-11.1501, -13.5877, -11.4622,  ..., -10.4687,  -9.8710,  -9.6263],
         [ -7.1713,  -7.8902, -10.6035,  ...,  -8.1884,  -6.4012,  -6.3499],
         ...,
         [-10.2086,  -9.9908, -10.0528,  ...,  -9.2802,  -9.6582,  -8.3069],
         [ -5.2291,  -6.5524,  -7.2077,  ...,  -5.9838,  -7.5163,  -5.9150],
         [ -7.1496,  -7.5715,  -7.7688,  ...,  -7.6387,  -6.2978,  -6.5950]]],
       grad_fn=<ViewBackward0>), hidden_states=None, attentions=None)
torch.Size([1, 13, 29794])
torch.Size([13, 29794])
tensor([  123,   231,   646,   183,   577, 13665,   123,  3827,   171,  1754,
          125,  2108,   123])


'O rato roeu a cabeça do rei de Roma'

In [39]:
ids = lm(**encoded).logits.squeeze(dim=0)
ids = ids.softmax(dim=1).argmax(dim=1)
print(ids)
tokenizer.decode(ids[1:-1])

tensor([  123,   231,   646,   183,   577, 13665,   123,  3827,   171,  1754,
          125,  2108,   123])


'O rato roeu a cabeça do rei de Roma'